# Technical note: Pacing

ODEs used in cardiac cellular electrophysiology are typically _forced_ by either applying a periodic stimulus current or by fixing the membrane potential to a stepwise constant value.
In Myokit, this is called _pacing_.

For efficient simulation, an _event-based pacing system_ is implemented in the Python classes `Protocol`, `ProtocolEvent`, and `PacingSystem`, and in the C header `pacing.h`.

In addition, a _fixed-form pacing system_ is implemented for the CVODE single-cell simulation only.
This is much less efficient, but may be useful in special cases.
Fixed-form pacing is implemented in the C header `pacing.h`.

For both systems, the output is a single valued, dimensionless quantity $x$, stored in the _pacing variable_.
Finally, simulations can be run without pacing.
Some notes on this are given in the section "No pacing".

## Time

The word _time_ in Myokit is used to denote a dimensionless clock variable  $t$, starting at $t=0$ and increasing indefinitely.

## Event-based pacing

Each event is defined as a 5-tuple $(x_i, t_i, d_i, p_i, m_i)$ or `(level, start, duration, period, multiplier)`.

Here,
- $x_i$ is a value for the pacing variable,
- $t_i$ is the time the event starts and
- $d_i$ is the event's duration.

The values $p_i$ and $m_i$ are used to create periodic events.
- When $p_i$ is set to any non-zero value, the event repeats every $p_i$ time units.
- The event then occurs $m_i$ times, unless $m_i$ is set to zero in which case it recurs indefinitely.

### Examples:

1. A pulse of $x=1$ for $0.5$ time units, starting at $t=50$ and recurring indefinitely with a period of $1000$ time units is specified as `(1, 50, 0.5, 1000, 0)`.

2. A singular pulse of $x=-40$, starting at $t=1000$ and lasting $100$ time units is specified as `(-40, 1000, 100, 0, 0)`.

3. A periodic pulse of $p=1$ with duration $d_i=0.5$ that starts at $t=20$, $t=1020$ and $t=2020$ is specified as `(1, 20, 0.5, 1000, 3)`.

### Definitions

- A singular event is an event with $p_i = 0$ and $m_i = 0$.
- A singular event $i$ is active when $t_i \leq t < t_i + d_i$.
- A periodic event is an event with $p_i > 0$.
- A periodic event with $m_i = 0$ occurs indefinitely.
- An indefinitely occuring periodic event $i$ is active when $ t_i + k \cdot r_i \leq t < t_i + k \cdot p_i + t_d$ where $k \in \{0,1,2,\ldots\}$.
- A periodic event with $m_i > 0$ occurs a total of $m_i$ times.
- A periodic event $i$ with $m_i > 0$ is active when $ t_i + k \cdot r_i \leq t < t_i + k \cdot p_i + t_d$ where $k \in \{0,1,2,\ldots,m_i-1\}$.

Restrictions:

- All events with $p_i = 0$ must have $m_i = 0$.
- The duration of a periodic event may never exceed its period: $d_i \leq p_i$.

<img src="figures/event.svg" width="500px" />

### Event-based pacing systems

A system is an ordered sequence of events.
It defines a pacing value $x$ and a special event, called the _current event_ that determines $x$'s value.
If there is no current event, then $x=0$.

Rules:

- No two events in a pacing system may start or re-occur at the same time.
- Whenever an event $i$ starts, it becomes the current event and the pacing variable's value changes to $x=x_i$.
- When a second event $j$ starts during, or just after the current event, it becomes the current event and $x=x_j$.
- When the current event deactivates, $x$ becomes zero, regardless of any previous events that may still be active.

Three examples of special cases are shown in the figure below:

<img src="figures/pacing-examples.svg" width="400px" />

Three examples of pacing:
_(Top)_ Three overlapping events.
_(Middle)_ Event 1 deactivates just when event 2 activates, the pacing level never goes down to zero but jumps from $x_1$ to $x_2$.
_(Bottom)_ An indefinitely re-occuring event with $d_i = p_i$: the pacing value stays at $x_i$ without interruption.

### Why event-based pacing

The two most common pacing methods used in cardiac cell simulation are (1) a periodic brief block current, or stimulus, and (2) a series of voltage steps.
Both cases fit neatly into the event-based scheme.

These protocols could also be implemented as part of the model - conceptually, they are outside the cell and therefore outside the model, but then again, so are the physical constants and external concentrations frequently included in cell models.

However, a typical stimulus current is a 0.5ms pulse given once every 1000ms.
For an efficient, variable step-size solver, this means there is a very large chance the solver will step over the stimulus completely.
A simple way to remedy this is to limit the maximum step size to less than 0.5ms, but (1) this requires the solver to know about the protocol and (2) this degrades the efficiency of the variable step-size scheme, especially in the diastolic phases where step sizes can easily exceed 100ms.
A much more efficient way to fix it is to tell the solver where the discontinuities are, so that it can ensure it doesn't step over them.
This is what the event-based protocol achieves.

## Fixed-form pacing

In real experiments, an "AP-clamp" or "data-clamp" protocol is sometimes employed during which a pre-calculated time-series signal is applied to the cell, either as a current or a voltage (see e.g. [Bebarova, 2012](https://doi.org/10.4149/gpb_2012_016)).
In Myokit single-cell simulations, there are several ways to achieve something similar:

- For piecewise constant signals, use the event-based pacing protocol. This is the most efficient method.
- For sinusoidal or other time-dependent, continuous signals, add equations to the model.
- For signals with a piecewise constant part and a more complicated part, split the simulation into two or more parts.
- If there is no other option, use the _fixed-form pacing_ capabilities of the `Simulation` class.
 
Fixed-form pacing allows you to specify a time series as a tuple $(\{t_0, t_1, ... , t_{n}\}, \{x_0, x_1, ... x_{n}\})$ where $t_i$ is a point in time and $x_i$ is the corresponding pacing value.
The times array may contain duplicates, but must be strictly non-decreasing.
At any time $t$, the value of the pacing variable $x$ is then defined by:

- Searching for indices $i$ and $i+1$ such that $t_i \leq t < t_{i+1}$.
- If such an interval can be found, the value is calculated from values $x_i$ and $x_{i+1}$ using linear interpolation.
- If the requested time is earlier than $t_0$, the value $x_0$ is used.
- If the requested time is later than $t_n$, the value $x_n$ is used.

Using bisection search, this can be implemented quickly.
However, (1) the interpolation is still only an approximation of the "true" value of whatever process generated the time series.
And (2), the interpolated function is not smooth: its derivatives may be discontinuities at any point $(t_i, x_i)$.

## No pacing

In the absence of a pacing protocol (event-based _or_ fixed-form) the pacing variable will have value $0$.

An alternative idea was to have the pacing variable be undefined.
In this case, any variables bound to the pacing variable would become unbound and retain their original, user-specified value.
_However,_ this would allow variables to change from bound to unbound without (A) changes to the model or (B) a change in simulation engine.
So by removing a protocol the result of running a simulation with `log=myokit.BOUND` would change.